In [18]:
import sounddevice as sd
import random
import numpy as np
import librosa

from threading import Thread
from pydub import AudioSegment
from scipy.io import wavfile
from midi2audio import FluidSynth
from midiutil import MIDIFile
from src.util.features import get_feature_vector
from scipy.spatial.distance import cosine, minkowski
from io import BytesIO

In [45]:
FONT = "./src/soundfonts/piano_eletro.sf2"
MIDI_FILE = "./src/input/temp"
WAV_FILE = "./src/output/temp"

In [42]:
def loop_play(file=None, sr=44100):
    t = Thread(target=play)
    t.start()

In [20]:
def play(file=None, sr=44100):
    if file == None: file=WAV_FILE+".wav"
    song, fs = librosa.load(file,sr=sr)
    sd.play(song, fs, loop=True)
    sd.wait()

In [21]:
def rand(low=0, high=2):
    return random.random()*(high - low + 1) + low

In [22]:
def save_harmony(midi_file, out):
    with open(MIDI_FILE + out + ".midi", "wb") as output_file:
        midi_file.writeFile(output_file)
        output_file.close()
    
    song = BytesIO()
    wav_song = BytesIO()
    
    midi_file.writeFile(song)

    FluidSynth(
            sound_font=FONT
        ).midi_to_audio(MIDI_FILE + out + ".midi", WAV_FILE + out + ".wav")

## Parameters to encode
 - [ ] Tempo
 - [ ] Nº of Notes
 - [x] Notes
 - [x] Notes duration
 - [x] Probability of operations 

## Methods For the Generation of the chromossome

In [41]:
MAX_NOTE = 108
MIN_NOTE = 21

In [49]:
def get_pitch(i: int):
    #21 -- 108
    #23 -- 75
    pitch_base = [23, 35, 47, 59, 71, 83, 95]
    return pitch_base[i]

In [50]:
def generate_random_note():
    p = random.choice(range(7))
    pitch = get_pitch(p)
    notes = [1, 3, 5, 6, 8, 10, 12]#list(range(1,12+1))
    
    note = random.choice(notes) + pitch
    return note

In [51]:
def generate_random_chord():
    base = generate_random_note()
    chord = [base, base+4, base+7]
    duration = random.choice(range(1, 8+1))
    return duration, chord

In [95]:
def generate_random_genome():
    N = 5
    probs = (0.4, #P(mutation)
             0.2, #P(crossover) 
             0.2, #P(inverse)
             0.2) #P(duplication)
    
    genome = [probs]
    for _ in range(N):
        duration, chord = generate_random_chord()
        melody_note = generate_random_note()
        genome.append( (duration, chord, melody_note) )
    
    return genome

In [118]:
def get_phenotype(genome):
    midi_file = MIDIFile(2)
    melody=0
    
    tempo = 120
    midi_file.addTempo(melody, 0, tempo)
    midi_file.addTempo(1,0,tempo)
    
    cum_time = 0
    
    probs, notes = genome[0], genome[1:]
    for el in notes:
        d, chord, note= el
        
        #for n in chord:
        #    midi_file.addNote(melody, 0, n, cum_time, d, 100)
        midi_file.addNote(melody, 0, note, cum_time, d/2, 100)
        
        cum_time+=d/2
    save_harmony(midi_file, "")
    return midi_file

In [97]:
gen = generate_random_genome()
gen

[(0.4, 0.2, 0.2, 0.2),
 (2, [29, 33, 36], 74),
 (6, [28, 32, 35], 88),
 (5, [41, 45, 48], 52),
 (2, [62, 66, 69], 69),
 (6, [55, 59, 62], 41)]

In [109]:
gen

[(0.4, 0.2, 0.2, 0.2),
 (2, [29, 33, 36], 74),
 (6, [28, 32, 35], 88),
 (5, [41, 45, 48], 52),
 (2, [62, 66, 69], 69),
 (6, [55, 59, 62], 41)]

In [145]:
gen_2=passing_tone(gen)
gen_2

[(0.4, 0.2, 0.2, 0.2),
 (2, [29, 33, 36], 74),
 (1.0, [29, 33, 36], 76),
 (1.0, [29, 33, 36], 77),
 (1.0, [29, 33, 36], 79),
 (1.0, [29, 33, 36], 81),
 (1.0, [29, 33, 36], 83),
 (1.0, [29, 33, 36], 84),
 (1.0, [29, 33, 36], 86),
 (6, [28, 32, 35], 88),
 (5, [41, 45, 48], 52),
 (2, [62, 66, 69], 69),
 (6, [55, 59, 62], 41)]

In [146]:
mf = get_phenotype(gen_2)

## Methods Regarding Mutation

In [33]:
def repeat(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    notes.insert(n, notes[n])
    genome = [probs, *notes]
    return genome

In [34]:
def split(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    duration, chord, note = notes[n]
    new_duration = duration/2
    
    new_note = (new_duration, chord, note)
    notes[n] = new_note
    notes.insert(n, new_note)
    
    genome = [probs, *notes]
    return genome

In [35]:
def arpeggiate(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    duration, chord, note = notes[n]
    pitch_incr = random.choice([4,7])
    
    new_pitch = min(MAX_NOTE, note+pitch_incr)
    new_note = (duration, chord, new_pitch)
    
    notes.insert(n+1, new_note)
    
    genome = [probs, *notes]
    return genome

In [60]:
def leap(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    duration, chord, note = notes[n]
    while (new:=generate_random_note())==note: continue
    
    new_note = (duration, chord, new)
    notes[n] = new_note
    
    genome = [probs, *notes]
    return genome

In [63]:
def get_pairs(notes):
    is_consecutive = []
    for i in range(len(notes)-1):
        d_a, chord_a, note_a = notes[i]
        d_b, chord_b, note_b = notes[i+1]
        
        if note_a==note_b:
            is_consecutive.append(i+1)
    return is_consecutive

In [66]:
def diatonic_upper_step_size(note):
    if note%12 in (0, 2, 5, 7, 9):
        return 2
    return 1

In [67]:
def diatonic_lower_step_size(note):
    if note%12 in (0, 5):
        return 1
    return 2

In [68]:
def upper_neighbor(genome):
    probs, notes = genome[0], genome[1:]
    
    is_consecutive = get_pairs(notes)
    if len(is_consecutive)==0:
        return genome
    
    n = random.choice(is_consecutive)
    d, chord, note = notes[n]
    
    step = diatonic_upper_step_size(note)
    note = min(MAX_NOTE, note+step)
    
    notes[n] = (d, chord, note)
    genome = [probs, *notes]
    return genome   

In [86]:
def lower_neighbor(genome):
    probs, notes = genome[0], genome[1:]

    is_consecutive = get_pairs(notes)
    if len(is_consecutive)==0:
        return genome
    
    n = random.choice(is_consecutive)
    d, chord, note = notes[n]
    
    step = diatonic_lower_step_size(note)
    note = max(MIN_NOTE, note - step)
    
    notes[n] = (d, chord, note)
    genome = [probs, *notes]
    return genome

In [89]:
def anticipation(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    duration, chord, note = notes[n]
    
    notes[n] = (duration * 0.25, chord, note)
    notes.insert(n+1, (duration * 0.75, chord, note))
    
    genome = [probs, *notes]
    return genome

In [92]:
def delay(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    duration, chord, note = notes[n]
    
    notes[n] = (duration * 0.75, chord, note)
    notes.insert(n+1, (duration * 0.25, chord, note))
    
    genome = [probs, *notes]
    return genome

In [143]:
def passing_tone(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice( range( len( notes ) - 1 ) )
    
    d, chord, note = notes[n]
    d_b, chord_b, note_b = notes[n+1]
    
    d = min(d, d_b)/2
    if note>note_b:   
        new_notes = []
        while note - ( step := diatonic_lower_step_size(note) ) > note_b:
            note -= step
            new = (d, chord, note)
            new_notes.append(new)
        notes = notes[:n+1] + new_notes + notes[n+1:]
        
    if note<note_b:
        new_notes = []
        while note + ( step := diatonic_upper_step_size(note) ) < note_b:
            note += step
            new = (d, chord, note)
            new_notes.append(new)
        notes = notes[:n+1] + new_notes + notes[n+1:]
    genome = [probs, *notes]
    return genome    

In [101]:
def delete_note(genome):
    probs, notes = genome[0], genome[1:]
    n = random.choice(range(len(notes)))
    
    notes = notes.copy()
    notes.pop(n)
    
    genome = [probs, *notes]
    return genome

In [102]:
def merge_note(genome):
    probs, notes = genome[0], genome[1:]
    is_consecutive = get_pairs(notes)
    
    if len(is_consecutive)==0: 
        return genome
    
    n = random.choice(is_consecutive)
    d_a, chord_a, note_a = notes[n-1]
    d_b, chord_b, note_b = notes[n]
    
    d = min(8, d_a + d_b)
    new = (d, chord_a, note_a)
    
    notes[n-1] = new
    notes = notes.copy()
    
    notes.pop(n)
    genome = [probs, *notes]
    return genome

In [139]:
def mutation(genome):
    rules = [
        repeat,
        split,
        arpeggiate,
        leap,
        upper_neighbor,
        lower_neighbor,
        anticipation,
        delay,
        passing_tone,
        delete_note,
        merge_note
    ]
    
    probs = np.ones(len(rules))/len(rules)
    mutation_rule = np.random.choice(
                        rules, size=1, p=probs
                    )[0]
    genome = mutation_rule(genome)
    return genome

## Methods Regarding Crossover

In [ ]:
def tournament(pop, k):
    sub_pop = np.random.choice(pop, size=k, replace=True)
    
    sub_fitness = list(map(fitness, sub_pop))
    
    

In [ ]:
def crossover():

In [48]:
sd.stop()

In [65]:
98%12

2